# Pynq-Z1 DES ACCELERATOR DEMO 
# HARDWARE API (HAPI)

* FULLY PARALLEL DES ACCELERATOR CODE
* Repository: https://github.com/IamVNIE/hapi-des
* Based-on: https://github.com/Xilinx/PYNQ

# Install package

In [ ]:
!sudo -H pip install --upgrade 'git+https://github.com/IamVNIE/hapi-des'

Step 1: Initialize DES ACCELERATOR

In [1]:
from hapiDES import hapiDES
from pynq import pl
from pynq import Overlay
des=hapiDES()
des.reset_des_accel()

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


DES ACCELERATOR RESET


Step 2: Initialize DES ACCELERATOR OPERATION -- SET TO ENCRYPTION

In [2]:
des.set_oper_encrypt()

DES ACCELERATOR IS SET TO ENCRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY


Step 3: Initialize ENCRYPTION KEY

In [3]:
MyKey=[0xfdcb0,0x167cde]
des.set_key(MyKey)

CRYPTO KEY UPDATED


Step 4: CHECK ENCRYPTION OPERATION ON SAMPLE DATA

In [4]:
input_data=[0xFE123410,0x12354567]
des.des_status()
des.encrypt(input_data)

DES ACCEL STATUS
ACTIVE (NOT IN RESET)
MODE:ENCRYPT


['0xdbd44175', '0x8cd6a970']

PERFORM OPERATION ON LARGE INPUT -- CAN INPUT A FILE

In [5]:
## This ia function to flatten the ASCII INPUT to HEX -- 
import time
import math
def data_formatter(txt):
    dataIn=''.join(hex(ord(x))[2:] for x in txt)
    print(dataIn)
    print(len(dataIn))
    vin=[]
    for i in range(0,len(dataIn),8):
        vin.append(''.join(map(str,dataIn[i:i+8])))
    vin=['0x' + s for s in vin]
    #print("INPUT")
    print(vin)
    vin_bc=vin
    vin=[int(x,16) for x in vin]
    if len(vin)%2!=0:
        vin.append(0)
    return vin,vin_bc

# Encryption on a Message

In [6]:
txt ="This is p a secret"
data_2send,enc_in_hex=data_formatter(txt)
#print(enc_in_hex)
cryptRes=[]
j=0
TOTAL_BLOCKS=len(data_2send)
start_time = time.time()
for i in range(0,len(data_2send),2):
    cryptRes[j:j+1]=des.encrypt(data_2send[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print("ENCRYPTED CIPHER TEXT")
print(cryptRes)

54686973206973206120736563726574
32
['0x54686973', '0x20697320', '0x61207365', '0x63726574']
--- 0.003149271011352539 seconds ---
--- 635.0676054205466 Blocks Per Second---
ENCRYPTED CIPHER TEXT
['0xe9d911cf', '0x75ee086f', '0x2ba28633', '0xb40f89f2']


# Decryption of encrypted message

In [7]:
des.set_oper_decrypt()
decipher=[]
j=0
TOTAL_BLOCKS=len(cryptRes)
print(cryptRes)
dec=(''.join(map(str,cryptRes)))
dec=dec.replace("0x","")
#print(dec)
ascii_string =''
x = 0
y = 2
l = len(dec)
while y <= l:
    ascii_string += chr(int(dec[x:y], 16))
    x += 2
    y += 2
print (ascii_string)
temp, temp2 = data_formatter(ascii_string)
print (temp)
cryptRes=[int(x,16) for x in cryptRes]
print(cryptRes)
start_time = time.time()
for i in range(0,len(cryptRes),2):
    decipher[j:j+1]=des.decrypt(cryptRes[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print(decipher)

DES ACCELERATOR IS SET TO DECRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY
['0xe9d911cf', '0x75ee086f', '0x2ba28633', '0xb40f89f2']
éÙÏuîo+¢3´ò
e9d911cf75ee86f2ba28633b4f89f2
30
['0xe9d911cf', '0x75ee86f2', '0xba28633b', '0x4f89f2']
[3923317199, 1978566386, 3123209019, 5212658]
[3923317199, 1978533999, 732071475, 3020917234]
--- 0.0013544559478759766 seconds ---
--- 1476.607639500088 Blocks Per Second---
['0x54686973', '0x20697320', '0x61207365', '0x63726574']


Verifying if we got the original input

In [ ]:
#bool(set(decipher) & set(enc_in_hex))
print(set(decipher)^set(enc_in_hex))
#print(x) for x,y in (decipher,enc_in_hex) if x!=y

# Print the message back in ascii

In [ ]:
dec=(''.join(map(str,decipher)))
dec=dec.replace("0x","")
print(dec)
ascii_string =''
x = 0
y = 2
l = len(dec)
while y <= l:
    ascii_string += chr(int(dec[x:y], 16))
    x += 2
    y += 2
print (ascii_string)

In [ ]:
print(str(2 + 2))

In [8]:
txt ="This is p a secret"
data_2send,enc_in_hex=data_formatter(txt)

546869732069732070206120736563726574
36
['0x54686973', '0x20697320', '0x70206120', '0x73656372', '0x6574']
